In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [2]:
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

In [3]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])

In [4]:
eclf3 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft', 
                         weights=[2,1,1], flatten_transform=True)

In [5]:
eclf3 = eclf3.fit(X, y)

In [6]:
print(eclf3.predict(X[0].reshape(1, -1)))

[1]


In [7]:
print(eclf3.transform(X[0].reshape(1, -1)))

[[8.65500004e-01 1.34499996e-01 1.00000000e+00 0.00000000e+00
  9.99999985e-01 1.52299840e-08]]


In [8]:
probs = eclf3.transform(X[0].reshape(1, -1))

In [9]:
for prob in probs[0]:
    print (prob)

0.8655000044537736
0.13449999554622638
1.0
0.0
0.9999999847700162
1.522998399037493e-08


In [10]:
from sklearn.pipeline import Pipeline

In [11]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

In [12]:
lr2 = Pipeline([('scaler', MinMaxScaler()), ('lr2', clf1)])

In [13]:
lr2.fit(X,y)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('lr2',
                 LogisticRegression(multi_class='multinomial',
                                    random_state=1))])

In [14]:
lr2[:1][0].transform(X[0].reshape(1, -1))

array([[0.33333333, 0.25      ]])

In [15]:
X[0].reshape(1, -1)

array([[-1, -1]])

In [16]:
lr2[:1][0]

MinMaxScaler()

In [17]:
lr2[-1:][0].predict(X[0].reshape(1, -1))

array([1])

In [18]:
lr2[:1]

Pipeline(steps=[('scaler', MinMaxScaler())])

In [19]:
lr2['scaler']

MinMaxScaler()

In [20]:
eclf4 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), ('lr2_MM', lr2)], voting='soft', 
                         weights=[2,1,1,1], flatten_transform=True)

In [21]:
eclf4 = eclf4.fit(X, y)

In [130]:
eclf4

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('rf',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=1)),
                             ('gnb', GaussianNB()),
                             ('lr2_MM',
                              Pipeline(steps=[('scaler', MinMaxScaler()),
                                              ('lr2',
                                               LogisticRegression(multi_class='multinomial',
                                                                  random_state=1))]))],
                 voting='soft', weights=[2, 1, 1, 1])

In [110]:
eclf4.estimators[0][0]

'lr'

In [22]:
probs = eclf4.transform(X[0].reshape(1, -1))

In [23]:
for prob in probs[0]:
    print (prob)

0.8655000044537736
0.13449999554622638
1.0
0.0
0.9999999847700162
1.522998399037493e-08
0.6204323678735139
0.37956763212648614


In [26]:
probs[0][::2] 

array([0.8655    , 1.        , 0.99999998, 0.62043237])

In [137]:
d = {}
for i in range(len(probs[0][::2])):
    print (i)
    print (eclf4.estimators[i][0])
    print (probs[0][::2][i])
    d[i] = {}
    d[i][eclf4.estimators[0][i]] = probs[0][::2][i]

0
lr
0.8655000044537736
1
rf
1.0
2
gnb
0.9999999847700162


IndexError: tuple index out of range

In [141]:
for i, model in enumerate(eclf4.estimators):
    print (model[0])
    print (probs[0][::2][i])
    d[i] = {}
    d[i][model[0]] = probs[0][::2][i]

lr
0.8655000044537736
rf
1.0
gnb
0.9999999847700162
lr2_MM
0.6204323678735139


In [142]:
d

{0: {'lr': 0.8655000044537736},
 1: {'rf': 1.0},
 2: {'gnb': 0.9999999847700162},
 3: {'lr2_MM': 0.6204323678735139}}

In [124]:
for i in range(3):
    print (i)

0
1
2


In [143]:
class ML_output:
    def __init__(self, models, x):
        self.x2 = x.reshape(1, -1)
        self.array = models.transform(self.x2)
        
        self.Model1Name = models.estimators[0][0]
        self.Model1Score = self.array[0][::2][0]
        self.Model1 = {self.Model1Name: self.Model1Score}
        
        self.NumberOfModels = len(self.array[0][::2])
        
        d = {}
        for i, model in enumerate(models.estimators):
            d[i] = {}
            d[i][model[0]] = self.array[0][::2][i]
        
        self.Models = d
        self.scores1 = self.array[0][::2]
        self.scores1.sort()
        
        self.FASE1 = min(self.scores1)
        self.FASE2 = self.scores1[-2]
        
        self.NumberOfModels = len(self.array[0][::2])

In [144]:
ML_output(eclf4,X[0].reshape(1, -1)).scores1

array([0.62043237, 0.8655    , 0.99999998, 1.        ])

In [146]:
ML_output(eclf4,X[0].reshape(1, -1)).Model1Name

'lr'

In [147]:
ML_output(eclf4, X[0]).Model1Score

0.8655000044537736

In [148]:
ML_output(eclf4,X[0].reshape(1, -1)).Model1

{'lr': 0.8655000044537736}

In [150]:
ML_output(eclf4,X[0].reshape(1, -1)).Models[0]

{'lr': 0.8655000044537736}

In [ ]:
ML_output(eclf4, X[0].reshape(1, -1)).FASE1

In [85]:
ML_output(eclf4, X[0].reshape(1, -1)).FASE1

0.6204323678735139

In [86]:
ML_output(eclf4, X[0].reshape(1, -1)).FASE2

0.9999999847700162

In [123]:
ML_output(eclf4, X[0]).NumberOfModels

4

In [28]:
ML_output(probs).scores

array([0.8655    , 1.        , 0.99999998, 0.62043237])

In [25]:
probs[0][1::2]

array([1.34499996e-01, 0.00000000e+00, 1.52299840e-08, 3.79567632e-01])

In [17]:
eclf4

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('rf',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=1)),
                             ('gnb', GaussianNB()),
                             ('lr2_MM',
                              Pipeline(steps=[('scaler', MinMaxScaler()),
                                              ('lr2',
                                               LogisticRegression(multi_class='multinomial',
                                                                  random_state=1))]))],
                 voting='soft', weights=[2, 1, 1, 1])

In [49]:
eclf4.estimators[0][0]

'lr'

In [81]:
eclf4.estimators[0]

('lr', LogisticRegression(multi_class='multinomial', random_state=1))

In [90]:
len(eclf4.estimators[0])

2

In [103]:
type(eclf4.estimators[0])

tuple

In [102]:
type(eclf4.estimators[3])

tuple

In [104]:
type(eclf4.estimators[3][0])

str

In [98]:
eclf4.estimators[3][1].steps

[('scaler', MinMaxScaler()),
 ('lr2', LogisticRegression(multi_class='multinomial', random_state=1))]

In [80]:
len(eclf4.estimators[3])

2

In [52]:
eclf4.estimators[0][1]

LogisticRegression(multi_class='multinomial', random_state=1)

In [77]:
eclf4.estimators[0][1].predict(X[0].reshape(1, -1))

array([1])

In [76]:
dir(eclf4.estimators[0][1])

['C',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_n_features',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_predict_proba_lr',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_data',
 'class_weight',
 'classes_',
 'coef_',
 'decision_function',
 'densify',
 'dual',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'intercept_scaling',
 'l1_ratio',
 'max_iter',
 'multi_class',
 'n_features_in_',
 'n_iter_',
 'n_jobs',
 'penalty',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'random_state',
 'score',
 'set_params',
 'solver',
 'sparsify',
 'tol',
 'verbose',
 'w

In [69]:
eclf4.estimators[3][1][:1][0]

MinMaxScaler()

In [70]:
eclf4.estimators[3][1][:1][0].transform(X[0].reshape(1, -1))

array([[0.33333333, 0.25      ]])

In [74]:
eclf4.estimators[3][1][-1:][0]

LogisticRegression(multi_class='multinomial', random_state=1)

In [75]:
dir(eclf4.estimators[3][1][-1:][0])

['C',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_n_features',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_predict_proba_lr',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_data',
 'class_weight',
 'classes_',
 'coef_',
 'decision_function',
 'densify',
 'dual',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'intercept_scaling',
 'l1_ratio',
 'max_iter',
 'multi_class',
 'n_features_in_',
 'n_iter_',
 'n_jobs',
 'penalty',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'random_state',
 'score',
 'set_params',
 'solver',
 'sparsify',
 'tol',
 'verbose',
 'w

In [72]:
eclf4.estimators[3][1][-1:][0].predict(X[0].reshape(1, -1))

array([1])

In [47]:
eclf4.estimators[3][1][-1:].steps[0][0]

'lr2'

In [107]:
for model in eclf4.estimators:
    print (model[0])
    print (type(model))
    print (type(model[0]))
    print (type(model[1]))

lr
<class 'tuple'>
<class 'str'>
<class 'sklearn.linear_model._logistic.LogisticRegression'>
rf
<class 'tuple'>
<class 'str'>
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
gnb
<class 'tuple'>
<class 'str'>
<class 'sklearn.naive_bayes.GaussianNB'>
lr2_MM
<class 'tuple'>
<class 'str'>
<class 'sklearn.pipeline.Pipeline'>


In [27]:
from sklearn.preprocessing import FunctionTransformer

In [30]:
def myScaler(x):
    x2 = x*2
    return x2

In [33]:
def pipelinize(function, active=True):
    return FunctionTransformer(function, validate=False, kw_args={'active':active})

In [36]:
def pipelinize2(function):
    return FunctionTransformer(function, validate=False)

In [37]:
myModel = Pipeline([
        ('my_scaler', pipelinize2(myScaler)),
        ('classifier', clf1)
    ])

In [38]:
test = myModel.fit(X,y)

In [39]:
test

Pipeline(steps=[('my_scaler',
                 FunctionTransformer(func=<function myScaler at 0x00000286F4E614C0>)),
                ('classifier',
                 LogisticRegression(multi_class='multinomial',
                                    random_state=1))])

In [40]:
myModel2 = Pipeline([
        ('my_scaler', pipelinize2(myScaler)),
        ('classifier', eclf4)
    ])

In [43]:
finalModel = myModel2.fit(X,y)

In [46]:
finalModel

Pipeline(steps=[('my_scaler',
                 FunctionTransformer(func=<function myScaler at 0x00000286F4E614C0>)),
                ('classifier',
                 VotingClassifier(estimators=[('lr',
                                               LogisticRegression(multi_class='multinomial',
                                                                  random_state=1)),
                                              ('rf',
                                               RandomForestClassifier(n_estimators=50,
                                                                      random_state=1)),
                                              ('gnb', GaussianNB()),
                                              ('lr2_MM',
                                               Pipeline(steps=[('scaler',
                                                                MinMaxScaler()),
                                                               ('lr2',
                                          

In [44]:
probs = finalModel.transform(X[0].reshape(1, -1))

In [45]:
probs

array([[9.38798074e-01, 6.12019260e-02, 1.00000000e+00, 0.00000000e+00,
        9.99999985e-01, 1.52299840e-08, 6.20432368e-01, 3.79567632e-01]])

In [31]:
x2 = myScaler(X)

In [32]:
x2

array([[-2, -2],
       [-4, -2],
       [-6, -4],
       [ 2,  2],
       [ 4,  2],
       [ 6,  4]])